<a href="https://colab.research.google.com/github/WHU-Peter/COMP6248-Deep-Learning/blob/master/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torchbearer
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# fix random seed for reproducibility
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

# flatten 28*28 images to a 784 vector for each image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert to tensor
    transforms.Lambda(lambda x: x.view(-1))  # flatten into vector
])

# load data
trainset = MNIST(".", train=True, download=True, transform=transform)
testset = MNIST(".", train=False, download=True, transform=transform)

# create data loaders
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=True)

# define baseline model
class BaselineModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(BaselineModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        if not self.training:
            out = F.softmax(out, dim=1)
        return out

In [ ]:
print(torch.cuda.list_gpu_processes())

cuda driver can't be loaded, is cuda enabled?


In [ ]:
# trainloader1 = DataLoader(trainset, batch_size=60000, shuffle=True)
# testloader1 = DataLoader(testset, batch_size=10000, shuffle=True)
# train_data = 0
# train_labels = 0
# test_data = 0
# test_labels = 0

# for inputs, labels in trainloader1:
#   train_data = inputs
#   train_labels = labels

# for inputs, labels in testloader1:
#   test_data = inputs
#   test_labels = labels

# train_data = train_data.to(device)
# train_labels = train_labels.to(device)
# test_data = test_data.to(device)
# test_labels = test_labels.to(device)
# print(train_data.shape)
# print(train_labels.shape)
# print(test_data.shape)
# print(test_labels.shape)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "cpu"
loss_function = nn.CrossEntropyLoss()


def training(model, epoches=10):
  # define the loss function and the optimiser
  optimiser = optim.Adam(model.parameters())
  model = model.to(device)

  train_loss_trace = np.zeros(epoches+1)
  train_accuracy_trace = np.zeros(epoches+1)
  test_loss_trace = np.zeros(epoches+1)
  test_accuracy_trace = np.zeros(epoches+1)
  # the epoch loop
  train_loss, train_accuracy, test_loss, test_accuracy = test(model)
  train_loss_trace[0] = train_loss
  train_accuracy_trace[0] = train_accuracy
  test_loss_trace[0] = test_loss
  test_accuracy_trace[0] = test_accuracy
  print("训练开始")
  print(train_loss, train_accuracy, test_loss, test_accuracy)
  for epoch in range(epoches):
      running_loss = 0.0
      for data in trainloader:
          # get the inputs
          inputs, labels = data
          inputs, labels = inputs.to(device), labels.to(device)

          # zero the parameter gradients
          optimiser.zero_grad()

          # forward + loss + backward + optimise (update weights)
          outputs = model(inputs)
          loss = loss_function(outputs, labels)
          loss.backward()
          optimiser.step()
      train_loss, train_accuracy, test_loss, test_accuracy = test(model)
      train_loss_trace[epoch+1] = train_loss
      train_accuracy_trace[epoch+1] = train_accuracy
      test_loss_trace[epoch+1] = test_loss
      test_accuracy_trace[epoch+1] = test_accuracy
      print("训练完成一次")
      print(train_loss, train_accuracy, test_loss, test_accuracy)
  fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
  ax[0,0].plot(train_loss_trace, c='m')
  ax[0,0].grid(True)
  ax[0,0].set_xlabel("Iteration", fontsize=14)
  ax[0,0].set_ylabel("Train_Loss", fontsize=14)
  ax[0,0].set_title("Traing Set", fontsize=14)
  ax[0,1].plot(test_loss_trace, c='r')
  ax[0,1].grid(True)
  ax[0,1].set_xlabel("Iteration", fontsize=14)
  ax[0,1].set_ylabel("Test_Loss", fontsize=14)
  ax[0,1].set_title("Test Set", fontsize=14)
  ax[1,0].plot(train_accuracy_trace, c='m')
  ax[1,0].grid(True)
  ax[1,0].set_xlabel("Iteration", fontsize=14)
  ax[1,0].set_ylabel("Train_Accuracy", fontsize=14)
  ax[1,0].set_title("Traing Set", fontsize=14)
  ax[1,1].plot(test_accuracy_trace, c='r')
  ax[1,1].grid(True)
  ax[1,1].set_xlabel("Iteration", fontsize=14)
  ax[1,1].set_ylabel("Test_Accuracy", fontsize=14)
  ax[1,1].set_title("Test Set", fontsize=14)
  plt.show()

def test(model):
  model.eval()
  # Compute the model accuracy on the test set
  correct = 0
  total = 0

  train_loss = 0
  train_correct = 0
  train_total = 0
  for data in trainloader:
    # get the inputs
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    outputs = model(inputs)
    train_loss += loss_function(outputs, labels)
    prediction = torch.argmax(outputs, 1)
    train_correct += (prediction == labels).sum().float()
    train_total += len(labels)
  train_accuracy = (100.0 * train_correct) / train_total
  
  test_loss = 0
  test_correct = 0
  test_total = 0
  for data in testloader:
    # get the inputs
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    outputs = model(inputs)
    test_loss += loss_function(outputs, labels)
    prediction = torch.argmax(outputs, 1)
    test_correct += (prediction == labels).sum().float()
    test_total += len(labels)
  test_accuracy = (100.0 * test_correct) / test_total
  model.train()
  return train_loss, train_accuracy, test_loss, test_accuracy

In [ ]:
model1 = BaselineModel(784, 784, 10)
training(model1)

model2 = BaselineModel(784, 10000, 10)
training(model2)

model3 = BaselineModel(784, 50000, 10)
training(model3)

model4 = BaselineModel(784, 100000, 10)
training(model4)

model5 = BaselineModel(784, 500000, 10)
training(model5)